# G1 scraping
This notebook aims to build a scaper of the news in https://g1.globo.com/

## Setup

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

## Start

In [3]:
driver.get('https://g1.globo.com/')